## 🎨 Monet like Painting 👩🏻‍🎨 with stable diffusion model 👨🏻‍💻

For more detailed overview, please checkout our [github](https://github.com/mohammadzainabbas/Deep-Learning-CS/) repo.

In [1]:
from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd, chdir

from dataclasses import dataclass, field
from typing import Union, List, Dict
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import logging as log
from enum import Enum

import base64

from IPython.display import display, Javascript

from dash import Dash, dcc, html, Input, Output
from dash.exceptions import PreventUpdate

import json

from jupyter_dash import JupyterDash
import plotly.express as px

import plotly.graph_objects as go

import plotly.tools as tls
from plotly.offline import plot, iplot

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Default configurations

In [2]:
#--------------------------
# some configuration(s)
#--------------------------
chdir("/Users/mohammadzainabbas/Masters/CS/Machine-Learning/Deep-Learning-CS")
data_dir = join(getcwd(), "data")
input_dir = join(data_dir, "raw", "input_images")
output_dir = join(data_dir, "raw", "output_images")

# get captions
with open(join(data_dir, "captions.json"), 'r') as f: 
   captions_dict = json.load(f)
   captions = [{"image": k, "caption": v} for k, v in captions_dict.items()]

# get input & output images
input_images = [x for x in listdir(input_dir) if isfile(join(input_dir, x))]
output_images = [x for x in listdir(output_dir) if isfile(join(output_dir, x))]

# @todo: temp usage - remove when we have output images for all input images
captions = [x for x in captions if x['image'] in output_images]

# others
is_cross_attention = False

### Demo

In [3]:
#------------------------------------
# Interactive component(s)
#------------------------------------
external_stylesheets = ['https://fonts.googleapis.com/css?family=Roboto:300,400,500,700&display=swap', 'https://fonts.googleapis.com/icon?family=Material+Icons']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

w_random = html.Button('Pick Random Picture', id='w_random')

w_cross_attention = html.Div([
    html.Label("Turn on cross-attention"),
    dcc.Checklist(
        id="w_cross_attention",
        options=[{"label": " ", "value": "show"}],
        value=["show"] if is_cross_attention else [],
        labelStyle={"display": "inline-block"}
    )
], style={"padding": "10px"})

w_loading = dcc.Loading(id="loading", children=[html.Div([html.Div(id="loading-output")])], type="circle")

app.layout = html.Div([
    html.H1("Demo"),
    w_random,
    html.Div([
        html.Div(id="w_output")
    ], style={})
], style={"background": "white","padding": "16px","display": "flex","flex-direction": "column"})


@app.callback(
    Output(component_id='w_output', component_property='children'),
    Input(component_id='w_random', component_property='n_clicks')
    )
def update_output(n_clicks):
    random_sample = np.random.choice(captions) # when we have saved output image for all the images
    image, caption = random_sample['image'], random_sample['caption']
    
    input_image = join(input_dir, image)
    output_image = join(output_dir, image)
    
    encoded_input_image = base64.b64encode(open(input_image, 'rb').read().decode('utf-8'))
    encoded_output_image = base64.b64encode(open(output_image, 'rb').read().decode('utf-8'))
    
    html_output = []
    
    html_output.append(html.Img(src='data:image/png;base64,{}'.format(encoded_input_image)))
    html_output.append(html.H5(f"Caption: '{caption}'"))
    html_output.append(html.Img(src='data:image/png;base64,{}'.format(encoded_output_image)))
    
    return html.Div(html_output)

In [4]:
app.run_server(mode='inline')

---------------------------------------------------------------------------
UnicodeDecodeError                        Traceback (most recent call last)
File /opt/homebrew/Caskroom/mambaforge/base/envs/deep_learning_project/lib/python3.10/codecs.py:322, in BufferedIncrementalDecoder.decode(
    self=<encodings.utf_8.IncrementalDecoder object>,
    input=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x...#\xb8 \x8f\xea\x7f:\xdapn6Z\x11\x19k\xa9\xff\xd9',
    final=True
)
    319 def decode(self, input, final=False):
    320     # decode input (taking the buffer into account)
    321     data = self.buffer + input
--> 322     (result, consumed) = self._buffer_decode(data, self.errors, final)
        data = b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b

In [5]:
# w_dataset = html.Div([
#     html.Label("Dataset"),
#     dcc.Dropdown(
#         id="w_dataset",
#         options=[{"label": x, "value": x} for x in data_files],
#         value=data_files[0],
#         clearable=False
#     )
# ], style={"padding": "10px"})

# w_algo_list = html.Div([
#     html.Label("Algorihtm(s)"),
#     dcc.Dropdown(
#         id="w_algo_list",
#         options=[{"label": convert_to_normal(x), "value": x} for x in algos],
#         value=algos[0],
#         multi=True,
#     )
# ], style={"padding": "10px"})

# w_sample_size = html.Div([
#     html.Label("Sample size"),
#     dcc.Input(
#         id="w_sample_size",
#         type="number",
#         value=sample_size,
#         min=1,
#         max=1000,
#         step=1
#     )
# ], style={"padding": "10px"})

# w_show_algo_comparsion = html.Div([
#     html.Label("Show algorithm comparsion"),
#     dcc.Checklist(
#         id="w_show_algo_comparsion",
#         options=[{"label": " ", "value": "show"}],
#         value=["show"] if show_algo_comparsion else [],
#         labelStyle={"display": "inline-block"}
#     )
# ], style={"padding": "10px"})

# w_show_dataset_comparsion = html.Div([
#     html.Label("Show dataset comparsion"),
#     dcc.Checklist(
#         id="w_show_dataset_comparsion",
#         options=[{"label": " ", "value": "show"}],
#         value=["show"] if show_dataset_comparsion else [],
#         labelStyle={"display": "inline-block"}
#     )
# ], style={"padding": "10px"})

# w_show_verbose = html.Div([
#     html.Label("Show verbose"),
#     dcc.Checklist(
#         id="w_show_verbose",
#         options=[{"label": " ", "value": "show"}],
#         value=["show"] if show_verbose else [],
#         labelStyle={"display": "inline-block"}
#     )
# ], style={"padding": "10px"})

# app.layout = html.Div([
#     html.H1("Centrality measures"),
#     html.Div([
#         w_dataset,
#         w_algo_list,
#         w_sample_size,
#         # w_show_algo_comparsion,
#         # w_show_dataset_comparsion,
#         # w_show_verbose
#     ], style={"padding": "16px"}),
#     html.Div([
#         html.Div(id="w_output")
#     ], style={})
# ], style={"background": "white","padding": "16px","display": "flex","flex-direction": "column"})

In [6]:
# @app.callback(
#     Output("w_output", "children"),
#     [
#         Input("w_dataset", "value"),
#         Input("w_algo_list", "value"),
#         Input("w_sample_size", "value"),
#         # Input("w_show_algo_comparsion", "value"),
#         # Input("w_show_dataset_comparsion", "value"),
#         # Input("w_show_verbose", "value")
#     ]
# )
# # def update_output(dataset, algo_list, sample_size, show_algo_comparsion, show_dataset_comparsion, show_verbose):
# def update_output(dataset, algo_list, sample_size):
    
    
#     data_file = join(data_dir, dataset)
#     show_verbose = False
    
#     graph = nx.read_edgelist(data_file, nodetype=str, data=(("weight", float),))
#     graph = sample_graph(graph=graph, sample_size=sample_size, verbose=show_verbose, plot=False)
#     di_graph = undirected_to_directed(graph)
    
#     algorithms = [x for x in algorithms_to_run if x.name in algo_list]
#     centrality_measures = compute_algorithms(graph=graph, algorithms=algorithms, verbose=show_verbose)

#     html_output = []

#     for i, centrality_measure in enumerate(centrality_measures):
#         html_output.append(
#             dcc.Graph(
#                 figure=plot_network_graph(graph=graph, centrality_result=centrality_measure),
#                 style={"width": "100%", "height": "100%"},
#                 id=f"graph_{i}_{centrality_measure.name}"
#             )
#         )

#     return html.Div(html_output)

In [7]:
# app.run_server(mode='inline')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'images' is not defined

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'images' is not defined

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'images' is not defined

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'images' is not defined

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'images' is not defined

-----------------------------------